## 컴파운드 패턴 개요

컴파운드 패턴은 2개 이상의 패턴을 합쳐 문제를 해결한다. 하지만 컴파운드 패턴은 단순히 여러 패턴의 조합이 아닌 문제를 해결하는 독립적인 솔루션이다.

## 모델-뷰-컨트롤러 패턴

MVC 패턴은 유저 인터페이스를 구현할 수 있는 유지보수가 용이한 디자인 패턴이다. MVC 패턴은 애플리케이션을 모델과 뷰, 컨트롤러로 나눠 구성한다. 각 파트는 맞물려 있으며 요청의 처리와 표현을 분리한다.

모델은 데이터와 비즈니스 로직(정보 저방 및 쿼리로직)을 처리하고 뷰는 데이터의 시각적 표현을 담당하며 컨트롤러는 사용자의 요청에 따라 모델과 뷰 사이에서 요청을 처리한다. 뷰와 컨트롤러는 모델에 의존하지만 그 반대는 아니다. 사용자가 데이터를 직접 요청하는 구조이기 때문이다. 모델의 독립성이 MVC 패턴의 중요한 부분이다.

In [1]:
class Model(object):
    services = {
        'email': {'number': 1000, 'price': 2,},
        'sms' : {'number': 1000, 'price': 10, },
        'voice': {'number': 1000, 'price': 15}
    }
    
class View(object):
    def list_services(self, services):
        for svc in services:
            print(svc, ' ')
    
    def list_pricing(self, services):
        for svc in services:
            print("For", Model.services[svc]['number'],
                     svc, 'message you pay $',
                     Model.services[svc]['price'])
            
class Controller(object):
    def __init__(self):
        self.model = Model()
        self.view = View()
        
    def get_services(self):
        services = self.model.services.keys()
        return(self.view.list_services(services))
    
    def get_priciing(self):
        services = self.model.services.keys()
        return(self.view.list_pricing(services))
    
class Client(object):
    controller = Controller()
    print("Services Provided:")
    controller.get_services()
    print("Pricing  for Services")
    controller.get_priciing()

Services Provided:
email  
sms  
voice  
Pricing  for Services
For 1000 email message you pay $ 2
For 1000 sms message you pay $ 10
For 1000 voice message you pay $ 15


## MVC 패턴 사용 사례

개발에서 자주 사용되는 많은 웹 애플리케이션 프레임워크가 MVC를 기반으로 한다.

토네이도 웹 애플맄에시녀 프레임워크를 사용해 단일 페이지 애플리케이션을 구현해본다. 사용자가 권한에 따라 일정을 생성, 수정, 삭제할 수 있는 일정 관리 프로그램을 작성한다.

* 컨트롤러부터 시작하자. 토네이도에서 컨트롤러는 뷰 또는 애플리케이션 라우팅에 해당된다. 일정 목록과 일정 생성, 종료, 에러 표시 등 여러 뷰가 필요하다.
* 모델은 일정을 조회와 생성, 삭제하는 데이터베이스 작업을 담당한다.
* 마지막으로 토네이도에서 뷰는 템플릿으로 표현된다. 일정을 조회와 생성, 삭제하는 템플릿과 잘못된 URL을 알리는 템플릿이 필요하다.

In [3]:
pip install tornado

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install db-sqlite3

    100% |████████████████████████████████| 174kB 1.8MB/s ta 0:00:01
  Stored in directory: /Users/hogeunryu/Library/Caches/pip/wheels/e0/f8/cb/08e3a3a63bdea9ac3a0cccd140636b6cf94e3fb7df1070b12e
  Stored in directory: /Users/hogeunryu/Library/Caches/pip/wheels/30/eb/ba/237fa002d1d1b2e73cedcefd26a9db37c4b72c7e5156ea0501
  Stored in directory: /Users/hogeunryu/Library/Caches/pip/wheels/b0/91/4d/f8fe808786ff1cda9e7e99e1b1bbda9196ab26786017965313
Successfully built db-sqlite3 db antiorm
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tornado
import tornado.web
import tornado.ioloop
import tornado.httpserver
import sqlite3

In [6]:
class IndexHandler(tornado.web.RequestHandler):
    def get(self):
        query = "select * from task"
        todos = _execute(query)
        self.render('index.html', todos=todos)
    
class NewHandler(tornado.web.RequestHandler):
    def post(self):
        name = self.get_argument('name', None)
        query = "create table if not exists tast (id INTEGER PRIMARY KEY, name TEXT, status NUMERIC)"
        _execute(query)
        query = "insert into task (name, status) values ('%s', %d) " %(name, 1)
        _execute(query)
        self.redirect('/')
        
    def get(self):
        self.render('new.html')
        
class UpdateHandler(tornado.web.RedirectHandler):
    def get(self, id, status):
        query = "update task set status=%d where id=%s " %(int(status), id)
        _execute(query)
        self.redirect('/')
        
class DeleteHandler(tornado.web.RequestHandler):
    def get(self, id):
        query = "delete from task where id=%s" % id
        _execute(query)
        self.redirect('/')

In [7]:
class RunApp(tornado.web.Application):
    def __init__(self):
        Handler = [
            (r'/', IndexHandler),
            (r'/todo/new', NewHandler),
            (r'/todo/update/(\d+)/status/(\d+)', UpdateHandler),
            (r'/todo/delete/(\d+)', DeleteHandler),
        ]
        
        settings = dict(
            debug=True,
            template_path='templates',
            static_path="static"
        )
        
        tornado.web.Application.__init__(self, Handler, **settings)

In [9]:
http_server = tornado.httpserver.HTTPServer(RunApp())
http_server.listen(5000)
tornado.ioloop.IOLoop.instance().start()

## MVC 패턴의 장점

* 애플리케이션의 모델과 뷰, 컨트롤러 총 3개의 파트로 나눌 수 있다. 이 구조는 유지보수가 쉽고 요소 간의 독립성이 높아져 복잡성이 줄어든다.
* 백엔드 로직을 거의 건드리지 않고 독립적으로 프론트엔드를 수정할 수 있다.
* 모델이나 비즈니스 로직도 마찬가지로 뷰와 상관없이 수정될 수 있다.
* 컨트롤러 또한 뷰와 모델과는 독립적으로 수정될 수 있다.
* 플랫폼 개발자와 UI 개발자 같이 특정 분야의 전문가들이 독립적으로 일할 수 있는 환경을 제공한다.

MVC는 웹사이트만 아니라 블로그나 영화 데이터베이스 앱, 비디오 스트리밍 앱 등 프로그램 내 요소 간의 높은 독립성을 요구하는 경우에 적합하다. 